In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff


# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
import yfinance as yf
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor


from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/Users/arturopolancolozano/opt/anaconda3/envs/finrl_env/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## 2.4. Create Folders

In [2]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



In [3]:
#TRAIN_START_DATE = '2001-01-01'
TRAIN_START_DATE = '2008-09-18'
TRAIN_END_DATE = '2019-01-31'
TRADE_START_DATE = '2019-02-01'
TRADE_END_DATE = '2019-06-30'

In [4]:
selected_companies_list = ["EC", "CIB", "ARGO"]
experiment_company = "ARGO"

ticker_dict = {
    "ARGO": "Cementos Argos", 
    "EC": "Ecopetrol",
    "CIB": "Bancolombia"    
}

experiment_company_name = ticker_dict[experiment_company]

print(f"Selected company: {experiment_company_name} with ticker: {experiment_company}")

Selected company: Cementos Argos with ticker: ARGO


In [5]:
argos_df = yf.download(
    "ARGO",
    TRAIN_START_DATE,
    TRADE_END_DATE,
).reset_index()
argos_df.columns= argos_df.columns.str.lower()
argos_df["tic"] = "ARGO"
argos_df.date = argos_df.date.astype(str)

bancolombia_df = yf.download(
    "CIB",
    TRAIN_START_DATE,
    TRADE_END_DATE,
).reset_index()
bancolombia_df.columns= bancolombia_df.columns.str.lower()
bancolombia_df["tic"] = "CIB"
bancolombia_df.date = bancolombia_df.date.astype(str)



ecopetrol_df = yf.download(
    "EC",
    TRAIN_START_DATE,
    TRADE_END_DATE,
).reset_index()
ecopetrol_df.columns= ecopetrol_df.columns.str.lower()
ecopetrol_df["tic"] = "EC"
ecopetrol_df.date = ecopetrol_df.date.astype(str)





[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [6]:
argos_df.head()

,date,open,high,low,close,adj close,volume,tic
0,2008-09-18,23.774214,27.223728,22.552509,26.282951,20.725945,536187,ARGO
1,2008-09-19,28.027309,28.190638,23.970209,27.073465,21.349312,756600,ARGO
2,2008-09-22,26.786007,28.144905,25.727633,25.727633,20.288036,108829,ARGO
3,2008-09-23,26.139221,26.701075,25.061249,25.361774,19.999533,187505,ARGO
4,2008-09-24,25.113514,26.126156,24.349133,24.394865,19.237057,186127,ARGO


In [7]:
argos_df.sort_values(['date','tic'],ignore_index=True)
bancolombia_df.sort_values(['date','tic'],ignore_index=True)
ecopetrol_df.sort_values(['date','tic'],ignore_index=True)

argos_df.head(3)

,date,open,high,low,close,adj close,volume,tic
0,2008-09-18,23.774214,27.223728,22.552509,26.282951,20.725945,536187,ARGO
1,2008-09-19,28.027309,28.190638,23.970209,27.073465,21.349312,756600,ARGO
2,2008-09-22,26.786007,28.144905,25.727633,25.727633,20.288036,108829,ARGO


In [8]:
prices = pd.DataFrame(
    {
        "Bancolombia": bancolombia_df["close"],
        "Ecopetrol": ecopetrol_df["close"],
        "Cementos Argos": argos_df["close"]

    }
)

prices.index = pd.to_datetime(ecopetrol_df.date)

fig = px.area(
    data_frame=prices.fillna(0),
    title = "Precio Cierre Acciones", 
    color_discrete_sequence=["yellow", "green", "grey"],
)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=10, label="10 Years", step="year", stepmode="backward"),
            dict(count=5, label="5 Years", step="year", stepmode="backward"),
            dict(count=1, label="1 Year", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.update_layout(
    xaxis_title='Tiempo',
    yaxis_title='Precio',
    legend_title="Acciones Disponibles",

)

fig.show()

In [10]:
prices.describe().T

,count,mean,std,min,25%,50%,75%,max
Bancolombia,2713.0,47.570509,12.722447,15.330000,38.709999,46.490002,58.779999,70.500000
Ecopetrol,2713.0,28.274807,16.323790,5.400000,15.230000,24.879999,41.950001,67.480003
Cementos Argos,2713.0,36.444014,16.400778,16.574657,20.775488,32.928493,50.332016,78.070000


In [9]:
returns_df = prices.pct_change().fillna(0.0)
returns_df.head()

,Bancolombia,Ecopetrol,Cementos Argos
date,,,
2008-09-18,0.000000,0.000000,0.000000
2008-09-19,0.031849,0.101010,0.030077
2008-09-22,-0.004315,-0.011009,-0.049710
2008-09-23,0.008333,-0.074212,-0.014220
2008-09-24,-0.049917,-0.020040,-0.038125


In [10]:

fig = make_subplots(
    rows=3, cols=1,
)

fig.add_trace(
    go.Scatter(
        x=returns_df.index, 
        y=returns_df["Cementos Argos"],
        name="Cementos Argos", 
        marker = {'color' : 'grey'}
        ),
    row=1, 
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=returns_df.index, 
        y=returns_df["Ecopetrol"],
        name="Ecopetrol", 
        marker = {'color' : 'green'}
        ),
    row=2, 
    col=1
)

fig.add_trace(
    go.Scatter(
        x=returns_df.index, 
        y=returns_df["Bancolombia"],
        name="Bancolombia", 
        marker = {'color' : 'gold'}
        ),
    row=3, 
    col=1, 
)

fig.update_layout(
    height=500, 
    width=850,
    title_text=f"Retorno Acciones"
)

In [11]:
fig = ff.create_distplot(
    [returns_df[c] for c in returns_df.columns], 
    returns_df.columns, 
    bin_size=.5, 
    colors = ["gold", "green", "grey"]
)
fig.update_layout(
    title=f'Distribución de los Retornos por Accion',
    xaxis_title='Retorno en %',
    yaxis_title='Densidad'
)

fig.show()

# Part 4: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

In [12]:
stocks_dict = {
    "EC": ecopetrol_df, 
    "CIB": bancolombia_df, 
    "ARGO": argos_df
}
feature_transformer = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = [],
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = feature_transformer.preprocess_data(stocks_dict[experiment_company])

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2712, 8)
Successfully added vix
Successfully added turbulence index


In [13]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

processed_full.sort_values(['date','tic'],ignore_index=True).head()

,date,tic,open,high,low,close,adj close,volume,vix,turbulence
0,2008-09-18,ARGO,23.774214,27.223728,22.552509,26.282951,20.725945,536187.0,33.099998,0.0
1,2008-09-19,ARGO,28.027309,28.190638,23.970209,27.073465,21.349318,756600.0,32.070000,0.0
2,2008-09-22,ARGO,26.786007,28.144905,25.727633,25.727633,20.288033,108829.0,33.849998,0.0
3,2008-09-23,ARGO,26.139221,26.701075,25.061249,25.361774,19.999533,187505.0,35.720001,0.0
4,2008-09-24,ARGO,25.113514,26.126156,24.349133,24.394865,19.237049,186127.0,35.189999,0.0


<a id='4'></a>
# Part 5. Build A Market Environment in OpenAI Gym-style
The training process involves observing stock price change, taking an action and reward's calculation. By interacting with the market environment, the agent will eventually derive a trading strategy that may maximize (expected) rewards.

Our market environment, based on OpenAI Gym, simulates stock markets with historical market data.

## Data Split
We split the data into training set and testing set as follows:

Training data period: 2009-01-01 to 2020-07-01

Trading data period: 2020-07-01 to 2021-10-31


In [14]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)

trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)

print(len(train))
print(len(trade))

2609
102


In [15]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=train.date, 
        y=train.close, 
        fill='tozeroy', 
        name = "Datos de Entrenamiento"
        )
) 

fig.add_trace(
    go.Scatter(
        x=trade.date, 
        y=trade.close, 
        fill='tozeroy', 
        name = "Datos de Evaluación", 
        marker = {'color' : 'blue'}

        )
) 

fig.update_layout(
    title=f'Partición Datos Entrenamiento y Evaluación para la accion {experiment_company_name}',
    xaxis_title='Tiempo',
    yaxis_title='Precio'
)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=10, label="10 Years", step="year", stepmode="backward"),
            dict(count=5, label="5 Years", step="year", stepmode="backward"),
            dict(count=1, label="1 Year", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()

In [16]:
stats = backtest_stats(train, value_col_name = 'close')

Annual return          0.092927
Cumulative returns     1.509231
Annual volatility      0.269651
Sharpe ratio           0.464541
Calmar ratio           0.239632
Stability              0.891963
Max drawdown          -0.387790
Omega ratio            1.090359
Sortino ratio          0.676348
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.069483
Daily value at risk   -0.033476
dtype: float64


In [17]:
stock_dimension = len(train.tic.unique())
#state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
state_space = 1 + 2*stock_dimension 

print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 1, State Space: 3


In [18]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

initial_budget = 1_000_000

env_kwargs = {
    "hmax": 500,
    "initial_amount": initial_budget,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,   
    #"tech_indicator_list": INDICATORS,
    "tech_indicator_list": [],

    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

timesteps_value = 500_000

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## Environment for Training



In [19]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Train DRL Agents
* The DRL algorithms are from **Stable Baselines 3**. Users are also encouraged to try **ElegantRL** and **Ray RLlib**.
* FinRL includes fine-tuned standard DRL algorithms, such as DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

### Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


### Agent 1: A2C


In [22]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

# set up logger
tmp_path = RESULTS_DIR + '/a2c_2'
new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
# Set new logger
#model_a2c.set_logger(new_logger_a2c)

trained_a2c = agent.train_model(
    model=model_a2c, 
    #tb_log_name='a2c_2',
    total_timesteps=timesteps_value
)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to results/a2c_2


TypeError: DRLAgent.train_model() missing 1 required positional argument: 'tb_log_name'

### Agent 2: DDPG

### Agent 3: PPO

Testing

In [ ]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)

df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c,
    environment = e_trade_gym
)



In [ ]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
df_actions_a2c = df_actions_a2c.set_index(df_actions_a2c.columns[0])
df_actions_a2c.actions = df_actions_a2c.actions.apply(lambda x: x[0])


In [ ]:
fig = px.area(
  data_frame=df_result_a2c,
  title = f"Resultado Portafolio Trading Bot {experiment_company_name}"
)

fig.add_hline(y=initial_budget, line_dash="dot")

fig.update_layout(
    xaxis_title='Tiempo',
    yaxis_title='Precio',
    showlegend=False
)


fig.show()


In [ ]:
trade_data = trade[["date", "close"]]
trade_data.set_index("date", inplace=True)

fig = px.area(
  data_frame=trade_data,
  title = f"Comportamiento Accion {experiment_company_name}"
)
fig.add_hline(
  y=trade_data.close.iloc[0], 
  line_dash="dot"
)

fig.update_layout(
    xaxis_title='Tiempo',
    yaxis_title='Precio',
    showlegend=False
)

fig.show()

In [ ]:


fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        f"Rendimiento Trading Bot", 
        f"Comportamiento Accion")
)

fig.add_trace(
    go.Scatter(
        x=df_result_a2c.index, 
        y=df_result_a2c.account_value,
        name="Bot"
        ),
    row=1, 
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=trade_data.index, 
        y=trade_data.close,
        name="Accion"
        ),
    row=1, 
    col=2
)



fig.update_layout(
    height=500, 
    width=850,
    title_text=f"Desempeño Trading Bot {experiment_company_name}"
)

fig.add_hline(
    y=initial_budget, 
    line_dash="dot", 
    col=1,
)

fig.add_hline(
    y=trade_data.close.iloc[0], 
    line_dash="dot", 
    col=2,
)

fig.show()

In [ ]:
df_actions_a2c.value_counts()

In [ ]:
df_actions_a2c_plot = df_actions_a2c.reset_index()
df_actions_a2c_plot.date = pd.to_datetime(df_actions_a2c_plot.date)
df_actions_a2c_plot.head(10)